# <font color='264CC7'> notion_uploader </font>

En construcción.

Los paquetes necesarios son:

In [ ]:
# !pip install pymupdf 
# !pip install openai
# !pip install notion-client
# !pip install pyTelegramBotAPI

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not install packages due to an OSError: ("Connection broken: OSError(28, 'No space left on device')", OSError(28, 'No space left on device'))



In [1]:
# Paquetes necesarios
import os
import fitz  # PyMuPDF
from openai import OpenAI 
from notion_client import Client
import telebot
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
from pydantic import BaseModel

In [2]:
# Asignar clave API de OpenAI
from keys import OPENAI_API_KEY, NOTION_TOKEN, NOTION_DATABASE_ID, TELEGRAM_TOKEN, USUARIO_AUTORIZADO

client = OpenAI(api_key=OPENAI_API_KEY)
notion = Client(auth=NOTION_TOKEN)
bot = telebot.TeleBot(TELEGRAM_TOKEN)

---
## <font color='264CC7'> Lectura del PDF y extracción de datos </font>


In [16]:
# 📄 Función: Extraer texto desde PDF
def extraer_texto_desde_pdf(ruta_pdf):
    texto = ""
    with fitz.open(ruta_pdf) as doc:
        for pagina in doc:
            texto += pagina.get_text()
    return texto

# Estructura de datos para Notion
class Articulo(BaseModel):
    titulo: str
    autor_principal: str
    revista: str
    año: int
    DOI: str
    tema: list[str]
    subtema: list[str]
    ideas_principales: list[str]
    citas_textuales: list[str]


# 🤖 Función: Analizar con GPT
def analizar_con_gpt(texto):
    prompt = """
Extrae la siguiente información del texto de un artículo académico:

- Título
- Autor principal
- Revista
- Año
- DOI
- Tema (un tema como: Educación, Matemática, Ciencia de Datos, etc.)
- Subtema (un subtema, máximo 2 palabras)
- Dos ideas principales
- Tres citas textuales

Texto:
""" + texto
    
    response = client.responses.parse(
        model="gpt-4o-mini",
        input=[{"role": "user", "content": prompt}],
        text_format = Articulo,
        temperature=0,
    )
    return response.output_parsed

# Crear post de LinkedIn
def crear_post_linkedin(articulo):
    prompt = f"""Crea un post de LinkedIn para compartir un artículo académico. El post debe ser atractivo, profesional y tener un formato similar a este:
    ---
    📚 ¿Cómo integrar ChatGPT en una clase de programación? 🤖💻

    Te recomiendo el artículo «Título», de autor principal. Presenta...

    Tres aspectos clave del estudio:
    1️⃣ 
    2️⃣ 
    3️⃣ 

    Una lectura fundamental para quienes experimentamos con metodologías activas y herramientas de IA en educación.

    👉 Accede al artículo completo aquí: DOI
    ---
    
    La información del artículo es la siguiente:
    - Título: {articulo.titulo}
    - Autor principal: {articulo.autor_principal}
    - Revista: {articulo.revista}
    - Año: {articulo.año}
    - DOI: {articulo.DOI}
    - Tema: {', '.join(articulo.tema)}
    - Subtema: {', '.join(articulo.subtema)}
    - Ideas principales: {', '.join(articulo.ideas_principales)}
    """

    response = client.responses.create(
        model="gpt-4o-mini",
        input=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )

    return response.output_text

In [13]:
ruta = "datos/test.pdf"
texto_extraido = extraer_texto_desde_pdf(ruta)
# Cantidad de caracteres extraídos
print(len(texto_extraido))

32994


In [14]:
json_data = analizar_con_gpt(texto_extraido)

print(json_data)

titulo='Minería de Datos en la Gestión Académica de la Educación Superior' autor_principal='Harold Elbert Escobar Terán' revista='Didasc@lia: Didáctica y Educación' año=2017 DOI='' tema=['Educación', 'Minería de Datos'] subtema=['Gestión Académica'] ideas_principales=['La minería de datos permite mejorar la eficiencia académica y la calidad del desempeño en instituciones educativas.', 'La implementación de técnicas de minería de datos ayuda a identificar patrones y problemas en el aprendizaje de los estudiantes.'] citas_textuales=['"La preocupación de los administradores educativos por mejorar la eficiencia académica... exige la implementación de procesos basados en el uso de tecnología e información."', '"La minería de datos educativos permite el desarrollo de métodos para el descubrimiento de tipos particulares de datos que provienen de plataformas educativas."', '"La implementación de minería de datos educativos (MDE), está ayudando a todos los agentes educativos que trabajan en dif

In [17]:
crear_post_linkedin(json_data)

'---\n📊 ¿Cómo puede la minería de datos transformar la gestión académica en la educación superior? 🎓✨\n\nTe recomiendo el artículo «Minería de Datos en la Gestión Académica de la Educación Superior», de Harold Elbert Escobar Terán. Presenta un enfoque innovador sobre cómo aplicar técnicas de minería de datos para optimizar procesos educativos.\n\nTres aspectos clave del estudio:\n1️⃣ Mejora de la eficiencia académica en instituciones educativas.\n2️⃣ Identificación de patrones y problemas en el aprendizaje de los estudiantes.\n3️⃣ Aumento de la calidad del desempeño académico mediante análisis de datos.\n\nUna lectura fundamental para quienes estamos comprometidos con la mejora continua en la educación a través de la tecnología.\n\n👉 Accede al artículo completo aquí: DOI\n---'

---
## <font color='264CC7'> Envío a Notion </font>

In [7]:
# json_data = """
# {
#     "titulo": "Minería de datos en la gestión académica de la educación superior",
#     "autor_principal": "Harold Elbert Escobar Terán",
#     "revista": "Didasc@lia: Didáctica y Educación",
#     "año": "2017",
#     "tema": "Educación",
#     "subtema": "Minería de datos",
#     "ideas_principales": [
#         "La minería de datos permite mejorar.",
#         "La implementación de técnicas de minería."
#     ],
#     "citas_textuales": [
#         "La minería de datos educativos.",
#         "El análisis de estos factores.",
#         "Estas técnicas aplicadas en los sistemas educativos tributa."
#     ]
# }"""

In [8]:
# 📤 Función: Enviar a Notion
def enviar_a_notion(datos):
    propiedades = {
        "Título": {"title": [{"text": {"content": datos.titulo}}]},
        "Autor principal": {"rich_text": [{"text": {"content": datos.autor_principal}}]},
        "Revista": {"rich_text": [{"text": {"content": datos.revista}}]},
        "Año": {"number": int(datos.año)},
        "Lectura": {"select": {"name": "Completa"}},
        "Tema": {"multi_select": [{"name": t.strip()} for t in datos.tema]},
        "Subtema": {"multi_select": [{"name": s.strip()} for s in datos.subtema]}
    }

    children = []

    if datos.ideas_principales:
        children.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "Ideas principales"}}]}
        })
        for idea in datos.ideas_principales:
            children.append({
                "object": "block",
                "type": "bulleted_list_item",
                "bulleted_list_item": {"rich_text": [{"text": {"content": idea}}]}
            })

    if datos.citas_textuales:
        children.append({
            "object": "block",
            "type": "heading_2",
            "heading_2": {"rich_text": [{"text": {"content": "Citas"}}]}
        })
        for cita in datos.citas_textuales:
            children.append({
                "object": "block",
                "type": "quote",
                "quote": {"rich_text": [{"text": {"content": cita}}]}
            })

    notion.pages.create(
        parent={"database_id": NOTION_DATABASE_ID},
        properties=propiedades,
        children=children
    )

In [9]:
enviar_a_notion(json_data)

---
## <font color='264CC7'> Bot de Telegram </font>

In [ ]:
# 📍 Restricción de acceso
def es_usuario_autorizado(message):
    return message.from_user.id == int(USUARIO_AUTORIZADO)

def acceso_restringido(func):
    def wrapper(message):
        if not es_usuario_autorizado(message):
            bot.reply_to(message, "⛔ No estás autorizado para usar este bot.")
            return
        return func(message)
    return wrapper

# /start
@bot.message_handler(commands=['start'])
@acceso_restringido
def start(message):
    bot.send_message(
        message.chat.id,
        "👋 ¡Hola! Envíame un artículo en formato PDF para procesarlo."
    )

# Archivos PDF
@bot.message_handler(content_types=['document'])
@acceso_restringido
def handle_document(message):
    if not message.document.file_name.lower().endswith(".pdf"):
        bot.reply_to(message, "Por favor, envía un archivo PDF.")
        return

    bot.send_message(message.chat.id, "📥 Descargando y procesando el archivo...")

    # Descargar el archivo
    file_info = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    ruta_pdf = f"datos/{message.document.file_name}"
    os.makedirs("datos", exist_ok=True)
    with open(ruta_pdf, 'wb') as new_file:
        new_file.write(downloaded_file)

    try:
        texto = extraer_texto_desde_pdf(ruta_pdf)
        json_data = analizar_con_gpt(texto)
        enviar_a_notion(json_data)
        post_linkedin = crear_post_linkedin(json_data)
        bot.send_message(message.chat.id, "✅ Artículo procesado y enviado a Notion.")
        bot.send_message(message.chat.id, "🔗 Post de LinkedIn generado:\n\n" + post_linkedin)
    except Exception as e:
        bot.send_message(message.chat.id, f"⚠️ Ocurrió un error: {str(e)}")

# Fallback: otros mensajes
@bot.message_handler(func=lambda message: True)
def fallback(message):
    if not es_usuario_autorizado(message):
        return  # silencio
    bot.reply_to(message, "Por favor, envía un archivo PDF.")

# Iniciar el bot
bot.polling()